In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [3]:
# Load API Credentials
with open('/Users/Melissa/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [4]:
# set our API call parameters 
LOCATION = 'Boise,ID'
TERM = 'Pizza'

# Create a results-in-progress JSON file, but only if it doesn't exist.

In [5]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_ID_pizza.json"
JSON_FILE

'Data/results_in_progress_ID_pizza.json'

# Check if our JSON_FILE already exists. This will prevent us from accidentally overwriting an existing file.

In [6]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_ID_pizza.json not found. Saving empty list to file.


# Determine how many results are already in the file

In [7]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


 # Figure out how many pages of results we will need

In [8]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [9]:
## How many results total?
total_results = results['total']
total_results

287

In [10]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [11]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

15

 # Add this page of results to .json file

In [12]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

# Set up a progress bar in our for loop.

In [13]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/15 [00:00<?, ?it/s]

# For Loop to call each page

In [14]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)

  0%|          | 0/15 [00:00<?, ?it/s]

  # Convert .json to dataframe

In [15]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,mabS8Eq8CLjMZdLl3V7GWA,guidos-original-new-york-style-pizza-downtown-...,Guido's Original New York Style Pizza Downtown,https://s3-media2.fl.yelpcdn.com/bphoto/uCmfN6...,False,https://www.yelp.com/biz/guidos-original-new-y...,313,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 43.6149556, 'longitude': -116.198...",[delivery],$,"{'address1': '235 N 5th St', 'address2': '', '...",+12083459011,(208) 345-9011,3611.749214
1,o7_5qYZan3jo9jtDUoHRsw,casanova-pizzeria-boise-2,Casanova Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/tt7Pdt...,False,https://www.yelp.com/biz/casanova-pizzeria-boi...,60,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 43.6204869, 'longitude': -116.220...",[delivery],$$,"{'address1': '2431 W Fairview Ave', 'address2'...",+12089952662,(208) 995-2662,2100.618313
2,3X2pmXKqaMQ6xZjJHlKn8w,the-wylder-boise-2,The Wylder,https://s3-media2.fl.yelpcdn.com/bphoto/yH9Q85...,False,https://www.yelp.com/biz/the-wylder-boise-2?ad...,480,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 43.6115114754023, 'longitude': -1...",[],$$,"{'address1': '501 W Broad St', 'address2': Non...",+12082093837,(208) 209-3837,3312.402056
3,-aZpjNPSNfTXYkLMkQDiKw,spitfire-pizza-boise,Spitfire Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/ydGrGp...,False,https://www.yelp.com/biz/spitfire-pizza-boise?...,243,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 43.58042, 'longitude': -116.2131}","[pickup, delivery]",$$,"{'address1': '2450 South Vista Ave', 'address2...",+12083429451,(208) 342-9451,4195.389528
4,VD4mZEBNiCeyGSq51XV9jw,3-wood-pizza-and-pub-boise,3 Wood Pizza & Pub,https://s3-media4.fl.yelpcdn.com/bphoto/sljIHF...,False,https://www.yelp.com/biz/3-wood-pizza-and-pub-...,134,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 43.545104, 'longitude': -116.312763}",[delivery],$$,"{'address1': '10497 W Lake Hazel Rd', 'address...",+12085934141,(208) 593-4141,9220.851731


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
282,6iU5g238EGFa9wBwuixIWg,chilis-meridian-5,Chili's,https://s3-media3.fl.yelpcdn.com/bphoto/BpRGCb...,False,https://www.yelp.com/biz/chilis-meridian-5?adj...,121,"[{'alias': 'tex-mex', 'title': 'Tex-Mex'}, {'a...",2.5,"{'latitude': 43.63312, 'longitude': -116.35356}","[delivery, pickup]",$$,"{'address1': '3078 N Eagle Rd', 'address2': No...",+12088845292,(208) 884-5292,9199.510084
283,lNQkuczvXv6vIT9JDacpHQ,applebees-grill-bar-garden-city-3,Applebee's Grill + Bar,https://s3-media1.fl.yelpcdn.com/bphoto/Dr9F5E...,False,https://www.yelp.com/biz/applebees-grill-bar-g...,47,"[{'alias': 'tradamerican', 'title': 'American ...",2.5,"{'latitude': 43.668344, 'longitude': -116.279842}","[delivery, pickup]",$$,"{'address1': '7025 N Glenwood St', 'address2':...",+12088532330,(208) 853-2330,6994.140937
284,PSy-g99cUhi0FRyTKmPRQg,taco-bell-eagle-3,Taco Bell,https://s3-media4.fl.yelpcdn.com/bphoto/MUuLLA...,False,https://www.yelp.com/biz/taco-bell-eagle-3?adj...,72,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",1.5,"{'latitude': 43.664327, 'longitude': -116.413251}","[delivery, pickup]",$,"{'address1': '6560 N. Linder Road', 'address2'...",+12088885333,(208) 888-5333,14877.396264
285,Dnlg9XT7FlruWq7B-V88Cw,red-robin-gourmet-burgers-and-brews-meridian,Red Robin Gourmet Burgers and Brews,https://s3-media1.fl.yelpcdn.com/bphoto/Y9a_VH...,False,https://www.yelp.com/biz/red-robin-gourmet-bur...,201,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",2.5,"{'latitude': 43.61845, 'longitude': -116.35545}","[delivery, pickup]",$$,"{'address1': '1475 N Eagle Rd', 'address2': ''...",+12084010184,(208) 401-0184,9041.356161
286,X8k7N03buo2bBbUUfcAltQ,taco-bell-meridian,Taco Bell,https://s3-media4.fl.yelpcdn.com/bphoto/LERjhE...,False,https://www.yelp.com/biz/taco-bell-meridian?ad...,66,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",1.5,"{'latitude': 43.59716, 'longitude': -116.39233}","[delivery, pickup]",$,"{'address1': '645 S Main St', 'address2': '', ...",+12088840091,(208) 884-0091,12077.209294


# Check for duplicates

In [16]:
# check for duplicate results
final_df.duplicated(subset='id').sum()

0

# Save the final DataFrame to a .csv (or a .csv.gz if it's too big for the GitHub file size limit).

In [17]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_ID_pizza.csv.gz', compression='gzip',index=False)